<a href="https://colab.research.google.com/github/Metallicode/Math/blob/main/Multi_Class_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MultiClass Logistic Regression

##Iris flower DataSet

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("iris.csv")

In [5]:
X = df.drop(["species"], axis=1)
y = df["species"]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  StandardScaler